***
*Project:* Helmholtz Machine on Niche Construction

*Author:* Jingwei Liu, Computer Music Ph.D., UC San Diego
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Experiment 3_4</span> 

# Real-Time Synthesis (Threading Debug)

*Created:* December 24, 2023

*Updated:* December 24, 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio
import pyaudio
import wave
import sys
import os
from pathlib import Path
import time
import threading
import logging

In [2]:
def intrument_choice(n_instr):
    # n_instr at most 8
    
    Instruments = [['kick','Tom'],['snare','clap'],['hihat closed','wood'],['ride','hihat open']]
    instr_choice = ['']*n_instr
    
    choice1_group = np.random.choice(4, 4, replace=False, p = [0.4, 0.3, 0.2, 0.1])
    choice_index = np.random.randint(2,size = 4)
    choice1_inst = choice_index[choice1_group]
    choice2_group = np.random.choice(4, 4, replace=False)
    choice2_inst = np.mod(choice_index+1,2)[choice2_group]
    choice = np.array([np.append(choice1_group,choice2_group),np.append(choice1_inst,choice2_inst)])
    
    for i in range(n_instr):
        instr_choice[i] = Instruments[choice[0,i]][choice[1,i]]
            
    return instr_choice

In [3]:
def y_init(division,BPM,fs):
    BS = 60/BPM # beat length in seconds
    len_gen = int(BS * fs * division + 0.5)
    y = np.zeros((len_gen*2,))
    return y,len_gen

In [46]:
def synthesis(data,division,instr_choice,instruments_sound,y,deviation):
    """
    Arguments:
    data -- generated single data point of length n, numpy array of shape (n,1)
    division -- number of beats per instrument track, integer number of 4,5,6
    instr_choice -- based on division and data length, we compute the number of instruments is n_instr = int(n/division+0.5), 
                    instr_choice is a list of chosen instrument names of shape length n_instr
    instruments_sound -- Python disctionary with key: instrument name & value: instrument wave form in floating number farmat [-1,1]
    y -- residual of previous generation for adding on, numpy array of shape (int(60/BPM * fs * division + 0.5)*2, )
    deviation -- deviation from beat grid in samples, a number likely 500-1000
    
    Returns:
    y -- generated audio in floating number farmat [-1,1], numpy array of shape (int(60/BPM * fs * division + 0.5)*2, )
    y_byte -- byte format of first half of y, namely y[:len(y)/2]
    """
    n = len(data)
    n_instr = int(n/division+0.5)
    y_beat = int(len(y)/8 + 0.5)
    
    for i in range(n_instr-1):
        notes = data[i*division:(i+1)*division]
        y_instr = instruments_sound[instr_choice[i]]
        for j in range(division):
            if notes[j] != 0:
                randomize = int(np.random.randn()*deviation)
                st = np.max([0,y_beat*j+randomize])
                instr_l = len(y[st:st+len(y_instr)])
                y[st:st+len(y_instr)] += y_instr[:instr_l]*np.random.rand()

    i = i+1
    notes = data[i*division:]
    k = len(notes)
    y_instr = instruments_sound[instr_choice[i]]
    for j in range(k):
        pos = np.random.choice(division, k, replace=False)
        if notes[j] != 0:
            randomize = int(np.random.randn()*deviation)
            st = np.max([0,y_beat*pos[j]+randomize])
            instr_l = len(y[st:st+len(y_instr)])
            y[st:st+len(y_instr)] += y_instr[:instr_l]*np.random.rand()
    
    y_max = np.max(np.abs(y))
    if y_max > 1:
        y = y/y_max*0.98
    
    # Int16 -- (-32,768 to +32,767)    
    y_int16 = (y[:int(len(y)/2)] * 32768 - 0.5).astype('int16')
    y_byte = y_int16.tobytes()
    
    return y, y_byte

In [47]:
def random_generate(k,n):
    
    u = np.random.rand(k,)
    c = np.random.randint(k, size=(n,1))
    mean = u[c]
    prob = np.random.randn(n,1) + mean
    random_gen = (prob>0.5).astype(int)
    
    return random_gen

In [48]:
# Preparation
n = 15
division = 4
n_instr = int(n/division+0.5)
instr_choice = intrument_choice(n_instr)

txt_folder = Path('Instruments').rglob('*.wav')
Instruments = []
for x in txt_folder:
    basename = os.path.basename(x)
    filename = os.path.splitext(basename)[0]
    Instruments.append(filename)
instruments_sound = {}
for i in range(len(Instruments)):
    instruments_sound[Instruments[i]],fs = librosa.load('Instruments/'+Instruments[i]+'.wav')

BPM = 120
deviation = 600

In [49]:
y,len_gen = y_init(division,BPM,fs)
data = random_generate(5,n)
y_out, y_byte = synthesis(data,division,instr_choice,instruments_sound,y,deviation)
y.fill(0)
y[:len_gen] = y_out[len_gen:]
y_play = y_out[:len_gen]
data

array([[0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0]])

In [50]:
data = random_generate(5,n)
y_out, y_byte_next = synthesis(data,division,instr_choice,instruments_sound,y,deviation)
y_play = np.append(y_play,y_out)
data

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1]])

In [51]:
y_play

array([ 0.02126914,  0.08391132, -0.10389422, ...,  0.        ,
        0.        ,  0.        ])

In [52]:
Audio(data=y_play, rate=fs)

In [53]:
Audio(data=y_out, rate=fs)

In [25]:
def streaming(threadname):
    global flag
    pya = pyaudio.PyAudio()
    stream = pya.open(format=pya.get_format_from_width(width=2), channels=1, rate=fs, output=True)
    logging.warning("Streaming start")
    while np.any(y_out!=0):
        stream.write(y_byte)
        flag = 1
        event.set()
        
        stream.write(y_byte_next)
        flag = 2
        event.set()
    stream.stop_stream()
    stream.close()
    pya.terminate()
    logging.warning("Streaming stop")

In [26]:
def control(threadname):
    global gen,y_out,y_byte, y_byte_next #, y_byte, y_byte_next, y_out
    while gen < 5:  # gen == True
        logging.warning("gen index: %d.", gen)
        data = random_generate(5,n)
        y_out, y_buffer = synthesis(data,division,instr_choice,instruments_sound,y,deviation)
        logging.warning("y_out is %f",np.sum(y_out))
        y.fill(0)
        y[:len_gen] = y_out[len_gen:]
        
        event.wait()
        if flag == 1:
            y_byte = y_buffer
            logging.warning("write y_byte")
        elif flag == 2:
            y_byte_next = y_buffer
            logging.warning("write y_byte_next")
        event.clear()
        gen += 1
    y_out.fill(0)
    logging.warning("Control thread finish")

In [33]:
# Threading
event = threading.Event()
flag = 0
gen = 0
thread1 = threading.Thread(target=streaming, args=("Thread-1", ) )
thread2 = threading.Thread(target=control, args=("Thread-2", ) )

thread1.start()
thread2.start()
thread1.join()
thread2.join()

In [34]:
stop = input("stop?")

stop?y


In [35]:
stop

'y'

### With user input

In [70]:
def streaming(threadname):
    global flag
    pya = pyaudio.PyAudio()
    stream = pya.open(format=pya.get_format_from_width(width=2), channels=1, rate=fs, output=True)
    logging.warning("Streaming start")
    while True:
        stream.write(y_byte)
        flag = 1
        event.set()
        if signal == 1:
            break
        
        stream.write(y_byte_next)
        flag = 2
        event.set()
        if signal == 2:
            break
    
    stream.stop_stream()
    stream.close()
    pya.terminate()
    logging.warning("Streaming thread finish")

In [71]:
def control(threadname):
    global y_out,y_byte, y_byte_next, signal #, y_byte, y_byte_next, y_out
    while stop == 'n':
        data = random_generate(5,n)
        y_out, y_buffer = synthesis(data,division,instr_choice,instruments_sound,y,deviation)
        y.fill(0)
        y[:len_gen] = y_out[len_gen:]
        
        event.wait()
        if flag == 1:
            y_byte = y_buffer
            logging.warning("write y_byte")
        elif flag == 2:
            y_byte_next = y_buffer
            logging.warning("write y_byte_next")
        event.clear()
        
    # after stop    
    y_int16 = (y[:int(len(y)/2)] * 32768 - 0.5).astype('int16')
    y_buffer = y_int16.tobytes()

    event.wait()
    if flag == 1:
        y_byte = y_buffer
        signal = 1
        logging.warning("write y_byte last")
    elif flag == 2:
        y_byte_next = y_buffer
        signal = 2
        logging.warning("write y_byte_next last")
    event.clear()
    logging.warning("Control thread finish")

In [72]:
def user_input(threadname):
    global stop
    stop = input("stop?(y)")
    logging.warning("User input thread finish")

In [73]:
# Threading
event = threading.Event()
stop = 'n'
signal = 0
thread1 = threading.Thread(target=streaming, args=("Thread-1", ) )
thread2 = threading.Thread(target=control, args=("Thread-2", ) )
thread3 = threading.Thread(target=user_input, args=("Thread-3", ) )

thread1.start()
thread2.start()
thread3.start()
thread1.join()
thread2.join()
thread3.join()

stop?(y)y
